In [1]:
import os

# JSON 키 파일의 경로를 환경 변수로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./keys/august-bond-397606-9fed2249633b.json"

In [2]:
import csv
from google.cloud import vision
from google.api_core.exceptions import ResourceExhausted

def process_image(client, image_url):
    """
    Google Cloud Vision API를 사용하여 이미지를 처리하고 관련 정보를 추출합니다.

    Args:
    - client: 초기화된 Google Cloud Vision 클라이언트.
    - image_url: 처리할 이미지의 URL.

    Returns:
    - 이미지에서 추출된 정보를 포함하는 딕셔너리.
    """
    # 이미지 객체를 생성하고 주어진 URL로 소스를 설정합니다.
    image = vision.Image()
    image.source.image_uri = image_url

    # 이미지에서 추출하려는 기능들을 정의합니다.
    features = [
        {"type_": vision.Feature.Type.TEXT_DETECTION},
        {"type_": vision.Feature.Type.FACE_DETECTION},
        {"type_": vision.Feature.Type.SAFE_SEARCH_DETECTION}
    ]

    # 이미지에 대한 정보를 추출하는 요청을 보냅니다.
    response = client.annotate_image({"image": image, "features": features})

    # 텍스트 데이터를 추출합니다.
    texts = response.text_annotations
    text_data = {"text": "", "area": 0, "area_percentage": 0, "char_count": 0}
    for text in texts[1:]:
        char_count = len(text.description)
        vertices = text.bounding_poly.vertices
        width = vertices[1].x - vertices[0].x
        height = vertices[3].y - vertices[0].y
        area = width * height
        area_percentage = round((area / 57600) * 100, 2)
        char_area = int(area / char_count)
        char_area_percentage = round((char_area / 57600) * 100, 2)

        if char_area_percentage >= 0.35:
            text_data["text"] += text.description + ' '
            text_data["area"] += area
            text_data["area_percentage"] += area_percentage
            text_data["char_count"] += char_count

    # 얼굴 감지
    has_face = 1 if response.face_annotations else 0

    # 유해한 콘텐츠 감지
    violence = response.safe_search_annotation.violence
    racy = response.safe_search_annotation.racy

    return {
        "text": text_data["text"].strip(),
        "char_count": text_data["char_count"],
        "total_area": text_data["area"],
        "total_area_percentage": text_data["area_percentage"],
        "char_area": int(text_data["area"] / text_data["char_count"]) if text_data["char_count"] > 0 else 0,
        "char_area_percentage": round(text_data["area_percentage"] / text_data["char_count"], 2) if text_data["char_count"] > 0 else 0,
        "has_face": has_face,
        "violence": violence,
        "racy": racy
    }

def detect_and_update_csv(csv_filename, debug_interval=50):
    """
    CSV 파일을 읽어 이미지 URL을 처리하고 결과를 CSV 파일에 업데이트합니다.

    Args:
    - csv_filename: 처리할 CSV 파일의 이름.
    - debug_interval: 디버그 메시지를 출력할 간격.
    """
    client = vision.ImageAnnotatorClient()

    # CSV 파일을 읽어 각 행을 리스트로 저장합니다.
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        rows = list(csv.DictReader(csvfile))
        fieldnames = rows[0].keys()

    # 이미지 URL을 키로 사용하여 각 행을 딕셔너리에 저장합니다.
    url_to_row_map = {row["thumbnail_url"]: row for row in rows}

    # 결과를 CSV 파일에 저장하는 함수
    def save_to_csv():
        # "_updated"를 추가하여 새로운 파일 이름을 생성합니다.
        new_csv_filename = csv_filename.replace('.csv', '_updated.csv')
        with open(new_csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csv_writer.writeheader()
            csv_writer.writerows(rows)

    # 각 이미지 URL을 처리합니다.
    for i, (image_url, row) in enumerate(url_to_row_map.items(), start=1):
        try:
            result = process_image(client, image_url)
            row.update(result)

            if i % debug_interval == 0:
                print(f"{len(url_to_row_map)}개의 썸네일 중 {i}개가 처리되었습니다")

        except ResourceExhausted:
            print(f"API 요청 제한으로 인해 작업이 중단되었습니다. 처리 중이던 썸네일: {image_url}")
            save_to_csv()
            return
        except Exception as e:
            print(f"처리 중이던 에러 썸네일: {image_url}")
            print(f"에러 메시지: {str(e)}")

    # 모든 이미지 처리가 완료된 후 결과를 CSV 파일에 저장합니다.
    save_to_csv()
    print("처리 완료되었습니다!")

detect_and_update_csv('part_2_230920_1144.csv')


13432개의 썸네일 중 50개가 처리되었습니다
13432개의 썸네일 중 100개가 처리되었습니다
13432개의 썸네일 중 150개가 처리되었습니다
13432개의 썸네일 중 200개가 처리되었습니다
13432개의 썸네일 중 250개가 처리되었습니다
13432개의 썸네일 중 300개가 처리되었습니다
13432개의 썸네일 중 350개가 처리되었습니다
13432개의 썸네일 중 400개가 처리되었습니다
13432개의 썸네일 중 450개가 처리되었습니다
13432개의 썸네일 중 500개가 처리되었습니다
13432개의 썸네일 중 550개가 처리되었습니다
13432개의 썸네일 중 600개가 처리되었습니다
13432개의 썸네일 중 650개가 처리되었습니다
13432개의 썸네일 중 700개가 처리되었습니다
13432개의 썸네일 중 750개가 처리되었습니다
13432개의 썸네일 중 800개가 처리되었습니다
13432개의 썸네일 중 850개가 처리되었습니다
13432개의 썸네일 중 900개가 처리되었습니다
13432개의 썸네일 중 950개가 처리되었습니다
13432개의 썸네일 중 1000개가 처리되었습니다
13432개의 썸네일 중 1050개가 처리되었습니다
13432개의 썸네일 중 1100개가 처리되었습니다
13432개의 썸네일 중 1150개가 처리되었습니다
13432개의 썸네일 중 1200개가 처리되었습니다
13432개의 썸네일 중 1250개가 처리되었습니다
13432개의 썸네일 중 1300개가 처리되었습니다
13432개의 썸네일 중 1350개가 처리되었습니다
13432개의 썸네일 중 1400개가 처리되었습니다
13432개의 썸네일 중 1450개가 처리되었습니다
13432개의 썸네일 중 1500개가 처리되었습니다
13432개의 썸네일 중 1550개가 처리되었습니다
13432개의 썸네일 중 1600개가 처리되었습니다
13432개의 썸네일 중 1650개가 처리되었습니다
13432개의 썸네일 중 1700개가 처리되었습니다
13432개의 썸네일 중 1750개가 처리되었습니다
13432